In [4]:
import cv2
import sys
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage,QPixmap
from PyQt5.QtWidgets import QApplication, QDialog
from PyQt5.uic import loadUi
from PyQt5.QtCore import QCoreApplication
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import model_from_json 


In [5]:
class Face_Recognition(QDialog):
    def __init__(self):
        super(Face_Recognition,self).__init__()
        loadUi('faceRecognition.ui',self)
        self.font=cv2.FONT_HERSHEY_SIMPLEX
        
        self.image=None
        self.sub_face=None
        self.loaded_model=None
        self.crop=None
        
        self.id=['Diaz','orang 1','orang 2','orang 3']
        self.startButton.clicked.connect(self.start_webcam)
        self.stopButton.clicked.connect(self.stop_webcam)
        self.saveButton.clicked.connect(self.save_img)
        
        self.textEdit.setText('not detected')
        self.cropButton.clicked.connect(self.crop_frame)
        
        self.detectButton.setCheckable(True)
        self.detectButton.toggled.connect(self.detect_face)
        self.faceEnabled=False
        
        self.faceCascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    def save_img(self):
        nama=self.textEdit2.toPlainText()
        cv2.imwrite(nama+'.jpg',self.crop)
        
    
    def detect_face(self,status):
        if status:
            self.detectButton.setText('stop deteksi')
            self.create_model()
            self.faceEnabled=True
        else:
            self.detectButton.setText('deteksi wajah')
            self.faceEnabled=False
    
    def create_model(self):
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        self.loaded_model.load_weights("model.h5")
        self.loaded_model.compile(optimizer = 'sgd',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
               
        
    def start_webcam(self):
        self.capture=cv2.VideoCapture(1)
        self.capture.set(3,480)
        self.capture.set(4,640)
        
        self.timer=QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(5)
        
    def update_frame(self):
        ret,self.image=self.capture.read()
#         self.image=cv2.flip(self.image,1)
        if(self.faceEnabled):
            detected=self.detect(self.image)
            self.displayImage(detected,1)
            
        else:
            self.displayImage(self.image,1)
            
    def detect(self,img):
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=self.faceCascade.detectMultiScale(gray,1.2,5,minSize=(90,90))
        for (x,y,w,h) in faces:
            self.sub_face=np.array(gray[y:y+h,x:x+w])
            self.sub_face=cv2.resize(self.sub_face,(100,100))
            citra=self.sub_face.reshape(1,100,100, 1)
            predict=self.loaded_model.predict_classes(citra)
            probabilitas=self.loaded_model.predict(citra)
            
            cv2.rectangle(img,(x,y),((x+w),(y+h)),(0,0,255),2)
            if (np.max(probabilitas)>0.76):
                self.textEdit.setText('terdeteksi :'+self.id[int(predict)])
            else:
                self.textEdit.setText('terdeteksi : Unknown')
        return img
    
    def crop_frame(self):
        self.crop=self.sub_face
        self.displayImage(self.sub_face,window=2)
        
        
    def stop_webcam(self):
        self.timer.stop()
        self.capture.release()
        cv2.destroyAllWindows()
        
    def displayImage(self,img,window=1):
        qformat=QImage.Format_Indexed8
        if len(img.shape)==3:
            if img.shape[2]==4:
                qformat=QImage.Format_RGBA8888
            else:
                qformat=QImage.Format_RGB888
                
        outImage=QImage(img,img.shape[1],img.shape[0],img.strides[0],qformat)
        outImage=outImage.rgbSwapped()
        
        if window==1:
            self.imgLabel.setPixmap(QPixmap.fromImage(outImage))
            self.imgLabel.setScaledContents(True)
        if window==2:
            self.imgLabel2.setPixmap(QPixmap.fromImage(outImage))
            self.imgLabel2.setScaledContents(True)
        

In [6]:
%tb
if __name__=='__main__':
#     app=QApplication(sys.argv)
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    window=Face_Recognition()
    window.setWindowTitle('faceRecognition')
    window.show()
    sys.exit(app.exec_())
    

SystemExit: 0

SystemExit: 0

C:\Users\DiazAgfaNuribi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
